 **Environment Setup**

Install Required Libraries

In [ ]:
!pip install transformers datasets torch torchaudio


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00


**Model and Dataset Preparation**

Load the Pre-trained Model

In [ ]:
from transformers import SpeechT5ForTextToSpeech, SpeechT5Processor

# Load pre-trained model and processor
model = SpeechT5ForTextToSpeech.from_pretrained('microsoft/speecht5_tts')
processor = SpeechT5Processor.from_pretrained('microsoft/speecht5_tts')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Dataset

In [ ]:
import pandas as pd
from datasets import Dataset

# Load your dataset from Excel file using pd.read_excel
df = pd.read_excel('/content/drive/MyDrive/DATASET FOR TTS/TTS-English.xlsx')
dataset = Dataset.from_pandas(df)

Dataset Preprocessing

In [ ]:
def preprocess_function(examples):
    # Ensure 'Text Example' is always a list of strings
    texts = examples['Text Example']

    # Check if 'texts' is already a list of strings
    if all(isinstance(t, str) for t in texts):
        pass  # Already in the correct format

    # If 'texts' is not a list of strings, attempt to convert it
    else:
        # Handle potential nested lists or empty lists
        texts = [str(t[0]) if isinstance(t, list) and len(t) > 0 else str(t) for t in texts]

        # If there are any non-string elements after conversion, raise an error with specifics
        non_string_elements = [t for t in texts if not isinstance(t, str)]
        if non_string_elements:
            raise ValueError(f"Non-string elements found in 'Text Example' after conversion: {non_string_elements}")

    inputs = processor(text=texts, return_tensors="pt", padding=True, truncation=True)

    # Convert each element in 'phonetic_transcription' to a string
    phonetic_transcriptions = [str(x) for x in examples['phonetic_transcription']]

    # Tokenize and add padding/truncation to 'phonetic_transcription'
    labels = processor.tokenizer(
        phonetic_transcriptions,
        return_tensors="pt",
        padding=True,  # Add padding to make all sequences the same length
        truncation=True  # Truncate sequences exceeding the maximum length
    )['input_ids']

    inputs['labels'] = labels
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

**Fine-tuning the Model**

Set Training Arguments

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./speechT5_finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,  # You can increase epochs if needed
    logging_dir='./logs',
    learning_rate=5e-5,
    save_steps=500,  # Saves the model every 500 steps
    evaluation_strategy="steps"
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Initialize Trainer

In [ ]:
import wandb
from transformers import Trainer, TrainingArguments

# Initialize WandB
wandb.login()

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    logging_dir='./logs',
    logging_steps=10,
    report_to="wandb",  # Enables logging to WandB
    ...

)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # Optional: you can create a separate evaluation set
    tokenizer=processor.tokenizer
)

# Start training
trainer.train()


**Evaluation**

Generate Speech from Text

In [ ]:
!pip install --upgrade torchaudio
!ffmpeg -i input_file.mp3 -ar 24000 -ac 1 -f s16 output_file.wav

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 685.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
import torchaudio
import torch
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

# Test input: A sentence with technical terms
test_sentence = "The API for CUDA is very efficient."

# Load the processor and model
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

# Pass the sentence as a keyword argument 'text'
inputs = processor(text=test_sentence, return_tensors="pt")

# Create dummy speaker embeddings (replace with actual embeddings if available)
speaker_embeddings = torch.zeros((1, 512))  # Make sure this shape matches the model's expectations

# Generate speech, including speaker embeddings
with torch.no_grad():  # Disable gradient calculation for inference
    generated_speech = model.generate(
        **inputs, speaker_embeddings=speaker_embeddings
    )

# Check the shape of the generated speech
print("Shape of generated speech:", generated_speech.shape)

# Check the data type of the generated speech
print("Data type of generated speech:", generated_speech.dtype)

# Convert to int16 format and scale appropriately
#generated_speech = generated_speech.cpu().type(torch.int16) #No need to cast to int16

# Reshape the tensor if necessary
# Assuming generated_speech has shape (100, 80) and you want mono audio
#generated_speech = generated_speech.view(-1) # Reshape to a 1D tensor
# Instead of view(-1), Reshape using view(1, -1) to add the channel dimension:

generated_speech = generated_speech.view(1, -1)  # Reshape to [channels, time]

# Save the generated speech with the appropriate sample rate
torchaudio.save(
    'generated_speech.wav',
    generated_speech.type(torch.float32),  # Save as float32
    24000,
    format='wav',
    #bits_per_sample=16,
    channels_first=True  # Set channels_first to True
)

print("Audio saved as 'generated_speech.wav'")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Shape of generated speech: torch.Size([104, 80])
Data type of generated speech: torch.float32
Audio saved as 'generated_speech.wav'


In [ ]:
!pip install git+https://github.com/suno-ai/bark

  Cloning https://github.com/suno-ai/bark to /tmp/pip-req-build-p0vtnwnr
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark /tmp/pip-req-build-p0vtnwnr
  Resolved https://github.com/suno-ai/bark to commit f4f32d4cd480dfec1c245d258174bc9bde3c2148
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 37.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 1.8 MB/s eta 0:00:00
  Created wheel for suno-bark: filename=suno_bark-0.0.1a0-py3-none-any.whl size=2567358 sha256=341d5a51bd0b1cd77a5fb03fa7ca9a8d04e988b52f4f8bb06f7bea17e3062afc
  Stored in directory: /tmp/pip-ephem-wheel-cache-7bnp6s8g/wheels/

Generate Audio from Text

In [ ]:
from bark import generate_audio, SAMPLE_RATE
import numpy as np
import soundfile as sf

# Define the text you want to convert into speech
text_prompt = "Hello, this is a test using the Bark library to generate speech!"

# Generate audio from text
audio_array = generate_audio(text_prompt)

# Save the audio to a WAV file
sf.write("output.wav", audio_array, SAMPLE_RATE)

print("Audio generated and saved as output.wav")


text_2.pt:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/bark/generation.py:212: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=device)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


coarse_2.pt:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

fine_2.pt:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:01<00:00, 64.2MB/s]
100%|██████████| 27/27 [11:42<00:00, 26.01s/it]


Audio generated and saved as output.wav


In [ ]:
import IPython.display as ipd
ipd.Audio("output.wav")

In [ ]:
import pandas as pd
import re
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the Excel file (update the file path if needed)
file_path = '/content/drive/MyDrive/DATASET FOR TTS/TTS-English.xlsx'
df = pd.read_excel(file_path)

# Extract the 'Text Example' column as a list
dataset = df['Text Example'].tolist()

# List of technical terms to look for in the dataset
technical_terms = ['API', 'CUDA', 'TensorFlow', 'PyTorch']

# Function to extract sentences with technical terms
def extract_sentences_with_terms(dataset, technical_terms):
    extracted_sentences = []
    for sentence in dataset:
        # Convert sentence to string if it's not already
        if not isinstance(sentence, str):
            sentence = str(sentence)
        for term in technical_terms:
            if re.search(r'\b' + term + r'\b', sentence):
                extracted_sentences.append(sentence)
                break
    return extracted_sentences

# Extract sentences containing technical terms
extracted_sentences = extract_sentences_with_terms(dataset, technical_terms)

# Display the extracted sentences
print(extracted_sentences)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['When you connect to an external service, make sure the API returns the correct status codes.', 'Developers need to understand the differences between REST and GraphQL for effective API design.', 'He explained how API gateways handle incoming requests and direct them to microservices.', ' The API call failed due to incorrect authentication.', 'In an interview, they asked me to design a RESTful API from scratch.', 'Understanding API rate limits is crucial when building applications that rely on external services.', 'Training the neural network on a GPU using CUDA reduced the training time by 50%.', ' The algorithm runs faster when parallelized across multiple CUDA cores.', 'To speed up the model inference, we moved from CPU computation to CUDA enabled GPUs.', 'The API returns a JSON response that needs to be parsed before displaying the data on the frontend.'

Generate Speech using Bark

In [ ]:
!pip install soundfile


In [4]:
!pip install git+https://github.com/suno-ai/bark.git
!pip install soundfile
import pandas as pd
import re
from google.colab import drive
from bark import generate_audio, SAMPLE_RATE
import soundfile as sf

# Mount Google Drive
drive.mount('/content/drive')

# Load the Excel file (update the file path if needed)
file_path = '/content/drive/MyDrive/DATASET FOR TTS/TTS-English.xlsx'
df = pd.read_excel(file_path)

# Extract the 'Text Example' column as a list
dataset = df['Text Example'].tolist()

# List of technical terms to look for in the dataset
technical_terms = ['API', 'CUDA', 'TensorFlow', 'PyTorch']

# Function to extract sentences with technical terms
def extract_sentences_with_terms(dataset, technical_terms):
    extracted_sentences = []
    for sentence in dataset:
        # Convert sentence to string if it's not already
        if not isinstance(sentence, str):
            sentence = str(sentence)
        for term in technical_terms:
            if re.search(r'\b' + term + r'\b', sentence):
                extracted_sentences.append(sentence)
                break
    return extracted_sentences

# Extract sentences containing technical terms
extracted_sentences = extract_sentences_with_terms(dataset, technical_terms)

# Display the extracted sentences
print(extracted_sentences)

# Generate and save audio for up to 5 sentences
for i, sentence in enumerate(extracted_sentences[:5]):  # Limit to 5 sentences
    audio = generate_audio(sentence) # Generate audio for the sentence using bark
    file_name = f"sentence_{i+1}.wav" # Create a file name for the audio
    sf.write(file_name, audio, SAMPLE_RATE) # Save the audio to a file using soundfile
    print(f"Generated and saved: {file_name}") # Print a confirmation message

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-ztiiqq10
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-ztiiqq10
  Resolved https://github.com/suno-ai/bark.git to commit f4f32d4cd480dfec1c245d258174bc9bde3c2148
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['When you connect to an external service, make sure the API returns the correct status codes.', 'Developers need to understand the differences between REST and GraphQL for effective API design.', 'He explained how API gateways handle incoming requests and direct them to microservices.', ' The API call failed due to incorrect authentication.', 'In an interview, they asked me to design a RESTful API from scratch.', 'Understanding API rate limits is crucial when building applications that rely on external services.', 'Training the neural network on a GPU using CUDA reduced the training time by 50%.', ' The algorithm runs faster when parallelized across multiple CUDA cores.', 'To speed up the model inference, we moved from CPU computation to CUDA enabled GPUs.', 'The API returns a JSON response that needs to be parsed before displaying the data on the frontend.'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


text_2.pt:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/bark/generation.py:212: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=device)


...

Generated and saved: sentence_3.wav


100%|██████████| 9/9 [02:26<00:00, 16.30s/it]


Generated and saved: sentence_4.wav


100%|██████████| 22/22 [07:23<00:00, 20.16s/it]


Generated and saved: sentence_5.wav


In [7]:
import pandas as pd
from google.colab import drive
import torchaudio
import IPython.display as ipd
import torch # Import torch explicitly

# Load the Excel file
file_path = '/content/drive/MyDrive/DATASET FOR TTS/TTS-English.xlsx'
df = pd.read_excel(file_path)

# Extract the 'Text Example' column as a list
dataset = df['Text Example'].tolist()

# Check if the 5th sentence exists in the dataset
sentence_index = 4  # Index for the 5th sentence (0-based indexing)
if len(dataset) > sentence_index:
    sentence = dataset[sentence_index]
    print(f"Selected sentence: {sentence}")

    # Load Bark and generate the audio
    from bark import generate_audio

    # Generate audio for the selected sentence
    audio_array = generate_audio(sentence)

    # Convert audio_array to a PyTorch tensor
    audio_array = torch.tensor(audio_array) # convert audio_array to tensor
    # If audio_array is 1D, add a dimension for channels
    if audio_array.dim() == 1:
        audio_array = audio_array.unsqueeze(0)  # Add channel dimension


    # Save the generated audio as a WAV file
    output_file = '/content/sentence_5.wav'
    torchaudio.save(output_file, audio_array, sample_rate=24000)

    # Play the generated audio
    ipd.display(ipd.Audio(output_file))

    print(f"Audio saved at: {output_file}")
else:
    print("The dataset doesn't have 5 sentences.")

Selected sentence: Integrating OAuth 2.0 with the existing system was a challenge, but it significantly improved security.


100%|██████████| 22/22 [07:20<00:00, 20.03s/it]


Audio saved at: /content/sentence_5.wav


**Evaluate Speech Quality**

Objective Metric: Mean Opinion Score (MOS)

In [8]:
# Example MOS collection
mos_scores = {
    'sentence_1.wav': [4, 5, 4],
    'sentence_2.wav': [3, 4, 4],
    # Add more scores for each audio sample
}

# Calculate the average MOS score for each audio
average_mos = {file: sum(scores)/len(scores) for file, scores in mos_scores.items()}
print(average_mos)

{'sentence_1.wav': 4.333333333333333, 'sentence_2.wav': 3.6666666666666665}
